* Tarea Arboles de regresión
* Sonia Mancera y Maximiliano Guerrero

In [291]:
# For Python 2 / 3 compatability
from __future__ import print_function
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.datasets import get_rdataset
from sklearn.model_selection import train_test_split

### data

In [292]:
hitters = get_rdataset('Hitters', 'ISLR').data
hitters = hitters.dropna()
# split 50-50 (asi estaba en R)
training_data, test_data = train_test_split(hitters, test_size=0.5, random_state=256)


In [293]:
hitters.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
rownames,,,,,,,,,,,,,,,,,,,,
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


## funciones

In [294]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, (int, float)) or (hasattr(value, 'dtype') and 'int' in str(value.dtype)) 
#le tuve que agregar esto porque me salia class numpy.int y no supe como mas arreglarlo. salu2

In [295]:
class Question:
    """A Question is used to partition a dataset."""
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = ">="
        if not is_numeric(self.value):
            condition = "=="
        return f"Is {self.column} {condition} {self.value}?"

In [296]:
def partition(df, question):
    """
    Divide un df según una pregunta.
    """
    true_rows = df[df.apply(lambda row: question.match(row), axis=1)]
    false_rows = df[~df.apply(lambda row: question.match(row), axis=1)]

    return true_rows, false_rows


In [297]:
def mse(df, objetivo):
    """
    Calcula el Error Cuadrático Medio (MSE) para una columna de un df.
    """
    valores_objetivo = df[objetivo]
    media = valores_objetivo.mean()
    mse_value = ((valores_objetivo - media) ** 2).mean()
    
    return mse_value


In [298]:
def info_gain(left, right, current_mse, objetivo):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_mse - p * mse(left, objetivo) - (1 - p) * mse(right, objetivo)

### demo

In [299]:
mi_pregunta = Question('CHits', 500)
mi_pregunta

Is CHits >= 500?

In [300]:
current_mse = mse(hitters, 'Salary')
current_mse

np.float64(202734.26915834734)

In [301]:
left_df, right_df = partition(hitters, mi_pregunta)
right_df.head(2)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
rownames,,,,,,,,,,,,,,,,,,,,
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [302]:
gain = info_gain(left_df, right_df, current_mse, 'Salary')
gain

np.float64(70605.39649291968)

In [303]:
def find_best_split(df, objetivo):
    """
    Encuentra la mejor pregunta para hacer dividiendo el DataFrame en función
    de la ganancia de información.
    """
    best_gain = 0 
    best_question = None  
    current_mse = mse(df, objetivo)  
    
    feature_columns = [col for col in df.columns if col != objetivo]

    for col in feature_columns:  

        values = df[col].unique() 
        for val in values: 
            question = Question(col, val)
            left_df, right_df = partition(df, question)
            if len(left_df) == 0 or len(right_df) == 0:
                continue

            gain = info_gain(left_df, right_df, current_mse, objetivo)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [304]:
#######
# Demo:
best_gain, best_question = find_best_split(training_data, 'Salary')
best_question
#######

Is CRuns >= 330?

In [305]:
class Leaf:
    """Un nodo de hoja clasifica datos.
    """

    def __init__(self, df, objetivo):
        self.prediction = df[objetivo].mean()
    
    def __repr__(self):
        """
        pa que si imprima
        """
        return f"Leaf(prediction={self.prediction})"



In [306]:
Leaf(hitters, 'Salary')


Leaf(prediction=535.9258821292775)

In [307]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [308]:
def build_tree(df, objetivo):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
        # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(df, objetivo)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 :
        return Leaf(df, objetivo)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    left_df, right_df = partition(df, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(left_df, objetivo)

    # Recursively build the false branch.
    false_branch = build_tree(right_df, objetivo)


    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [309]:
def build_tree(df, objetivo, depth=0, max_depth=4):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Try partitioning the dataset on each of the unique attributes,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(df, objetivo)

    # Base case: no further info gain or max depth reached
    # Since we can ask no further questions or have reached max depth,
    # we'll return a leaf.
    if gain == 0 or depth >= max_depth:
        return Leaf(df, objetivo)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    left_df, right_df = partition(df, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(left_df, objetivo, depth + 1, max_depth)

    # Recursively build the false branch.
    false_branch = build_tree(right_df, objetivo, depth + 1, max_depth)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)


In [310]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.prediction)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

## build a tree

In [311]:
my_tree = build_tree(training_data, 'Salary')

In [312]:
print_tree(my_tree)

Is CRuns >= 330?
--> True:
  Is Runs >= 84?
  --> True:
    Is PutOuts >= 292?
    --> True:
      Is CHmRun >= 93?
      --> True:
        Predict 1936.493333333333
      --> False:
        Predict 1020.0
    --> False:
      Is Assists >= 290?
      --> True:
        Predict 704.28575
      --> False:
        Predict 1210.4167499999999
  --> False:
    Is PutOuts >= 295?
    --> True:
      Is CHmRun >= 194?
      --> True:
        Predict 1183.3333333333333
      --> False:
        Predict 798.1818181818181
    --> False:
      Is Hits >= 119?
      --> True:
        Predict 732.4545454545455
      --> False:
        Predict 504.31370588235296
--> False:
  Is CRuns >= 211?
  --> True:
    Is PutOuts >= 109?
    --> True:
      Is Division == E?
      --> True:
        Predict 638.3332857142858
      --> False:
        Predict 486.458375
    --> False:
      Is Years >= 10?
      --> True:
        Predict 276.25
      --> False:
        Predict 393.75
  --> False:
    Is AtBat >= 151

## predictions

In [322]:
def predict(row, node):
    """Recursively predict the value for a given row using the decision tree."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.prediction

    # Decide whether to follow the true-branch or the false-branch.
    if node.question.match(row):
        return predict(row, node.true_branch)
    else:
        return predict(row, node.false_branch)


In [336]:
#######
# Demo:
# The tree predicts the 1st row of our data is
predict(test_data.iloc[0], my_tree)
#######

np.float64(121.84482758620689)

In [330]:
def print_leaf(predictions):
    """A nicer way to print the predictions at a leaf for regression."""
    return f"Predicted value: {predictions:.2f}"


In [338]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(predict(test_data.iloc[0], my_tree))
#######

'Predicted value: 121.84'

In [337]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(predict(test_data.iloc[1], my_tree))
#######

'Predicted value: 638.33'

In [339]:
def print_prediction_results(df, tree, objetivo):
    """Print actual and predicted values for each row in the DataFrame."""
    for index, row in df.iterrows():
        actual = row[objetivo]  # Get the actual value from the target column
        predicted = predict(row, tree)  # Predict the value using the decision tree
        print("Actual: {:.2f}. Predicted: {:.2f}".format(actual, predicted))

# Example usage:
print_prediction_results(test_data, my_tree, 'Salary')  # Replace 'Salary' with your target column


Actual: 135.00. Predicted: 121.84
Actual: 525.00. Predicted: 638.33
Actual: 1100.00. Predicted: 504.31
Actual: 100.00. Predicted: 121.84
Actual: 100.00. Predicted: 121.84
Actual: 250.00. Predicted: 638.33
Actual: 450.00. Predicted: 504.31
Actual: 1183.33. Predicted: 798.18
Actual: 1043.33. Predicted: 732.45
Actual: 225.00. Predicted: 504.31
Actual: 415.00. Predicted: 275.07
Actual: 1500.00. Predicted: 732.45
Actual: 787.50. Predicted: 1936.49
Actual: 475.00. Predicted: 486.46
Actual: 67.50. Predicted: 121.84
Actual: 215.00. Predicted: 121.84
Actual: 773.33. Predicted: 732.45
Actual: 750.00. Predicted: 504.31
Actual: 75.00. Predicted: 121.84
Actual: 740.00. Predicted: 504.31
Actual: 800.00. Predicted: 504.31
Actual: 100.00. Predicted: 504.31
Actual: 155.00. Predicted: 121.84
Actual: 850.00. Predicted: 504.31
Actual: 500.00. Predicted: 732.45
Actual: 165.00. Predicted: 121.84
Actual: 550.00. Predicted: 638.33
Actual: 1310.00. Predicted: 1936.49
Actual: 1220.00. Predicted: 638.33
Actual: 